In [ ]:
# Load the data. 
data <- read.csv("link")
head(data)

In [ ]:
# For synthetic control analysis in R, we can use the Synth package.
# If you haven't installed the package already, you can install it by
# running the command install.package("Synth")
# Now, we can load the package
library(Synth)

# The synth() function we're going to use next takes a standard panel dataset.
# So we first need to change the dataframe into a panel dataset by using the 
# function dataprep().
# The 'foo' argument takes the dataset. 
# The 'predictors' argument takes the vector of all predictors. 
# The argument 'predictors.op' can take different methods used for matching 
# predictors. If we use 'mean', we use mean of the predictors.
# If there are any additional numeric predictors, we feed them into the argument
# 'special.predictors'. Here we are using the pre-treatment values of the outcome
# variable as well. 
# We then have to define the dependent variables using the argument 'dependent'. 
# Our dependent variable is CO2 emissons per capital from transportation.
# In argument 'unit.variable' we enter the column number or the column name 
# of the column that identifies the donor pool unit numbers. 
# The argument 'unit.names.variable' takes the column number or the column name 
# of the column that identifies the donor pool unit names.
# The argument 'time.variable' takes the column number or the column name of the
# column that represents time periods.
# So far we haven't told the function which unit is our treatment unit. We can 
# do that using the argument 'treatment.identifier'. Unit 13 is Sweden. 
# The rest of the units are the control units. We define them using the argument
# 'controls.identifier'. 
# 'time.optimize.ssr' defines the periods of the time interval over which we want
# minimze the objective function. This is usually set at the pre-treatment time
# interval. Finally, 'time.plot' is the argument that sets the time interval over
# which the results of the analysis are to be plotted. This usually is both the
# pre-treatment and post-treatment time intervals combined. 
dataprep.out <-
    dataprep(foo = data,
             predictors = c("predictor1" , "predictor2" , "predictor3" ,
                            "predictor4") ,
             predictors.op = "mean" ,
             time.predictors.prior = 1960:2010 ,
             special.predictors = list(
                 list("outcomevar" , 2010 , "mean"),
                 list("outcomevar" , 2000 , "mean"),
                 list("outcomevar" , 1990 , "mean")
             ),
             dependent = "outcomevar",
             unit.variable = "countryno",
             unit.names.variable = "country",
             time.variable = "year",
             treatment.identifier = 13,
             controls.identifier = c(1:12, 14:15),
             time.optimize.ssr = 1960:2010,
             time.plot = 1960:2020
    )

In [ ]:
# The function synth() helps us find the weights using the algorithm we discussed.
# We have to tell the function what our panel dataset is. 
synth.out <- synth(data.prep.obj = dataprep.out)

# The weigts are stored in the output object. This is how we can retrieve 
# country weights in the synthetic Sweden.
# We can see that Denmark has the highest weight.
synth.tables <- synth.tab(dataprep.res = dataprep.out,
                          synth.res = synth.out
)
synth.tables$tab.w[1:14, ]

In [ ]:
# We can also look at the pre-treatment mean of the covariates for 
# the synthetic Sweden and the treated unit. You can see that the values
# are pretty similar indicating that the weights are well-calculated.
synth.tables$tab.pred[1:7, ]

In [ ]:
# The package has a function for creating path plots based on the results.
path.plot(synth.res = synth.out,
          dataprep.res = dataprep.out,
          Ylab = "Outcome Variable",
          Xlab = "Year",
          Ylim = c(0,3),
          Legend = c("Country","synthetic Country"),
          Legend.position = "bottomright"
)

In [ ]:
# The function gap.plot() plots the gaps between the potential outcome of the 
# treated unit and the synthetic unit.
gaps.plot(synth.res = synth.out,
          dataprep.res = dataprep.out,
          Ylab = "outcome variable",
          Xlab = "Year",
          Ylim = c(-0.5,0.5),
          Main = NA
)

In [ ]:
# In R, you can use the outcome of the synth() function from the package Synth
# to calculate MSPE. Unfortunately, the package doesn't spit out MSPE automatically.
# The line below, estimates the gap between the treated and the synthetic units
gaps <- dataprep.out$Y1plot - (dataprep.out$Y0plot %*% synth.out$solution.w)

# This calculates MSPE for the post-treatment period
mspepost <- mean((gaps[31:46, 1])^2)

# This calculates MSPE for the pre-treatment period
mspepre <- mean((gaps[1:30, 1])^2)

# Therefore, the ratio is simply:
msperatio = mspepost/mspepre

# You can use the code above in a for-loop in order to find the ratio for each
# control unit in the donor pool.